In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, models
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split



In [ ]:
# Define a custom dataset
class VideoDataset(Dataset):
    def __init__(self, real_dir, fake_dir, transform=None, num_frames=16):
        self.real_dir = real_dir
        self.fake_dir = fake_dir
        self.transform = transform
        self.num_frames = num_frames

        # Get list of video paths and labels
        self.real_videos = [os.path.join(real_dir, fname) for fname in os.listdir(real_dir)]
        self.fake_videos = [os.path.join(fake_dir, fname) for fname in os.listdir(fake_dir)]
        self.video_paths = self.real_videos + self.fake_videos
        self.labels = [0] * len(self.real_videos) + [1] * len(self.fake_videos)

    def __len__(self):
        return len(self.video_paths)

    def __getitem__(self, idx):
        video_path = self.video_paths[idx]
        frames = self.extract_frames(video_path)
        label = torch.tensor(self.labels[idx], dtype=torch.float32)
        if self.transform:
            frames = [self.transform(frame) for frame in frames]
        frames = torch.stack(frames)
        return frames, label

    def extract_frames(self, video_path):
        cap = cv2.VideoCapture(video_path)
        frames = []
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        frame_indices = np.linspace(0, max(1, total_frames) - 1, self.num_frames, dtype=int)
        
        for i in frame_indices:
            cap.set(cv2.CAP_PROP_POS_FRAMES, i)
            ret, frame = cap.read()
            if ret:
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            else:
                frame = np.zeros((224, 224, 3), dtype=np.uint8)  # Padding with black frames
            frames.append(frame)
        cap.release()
        return frames

# Use ResNet-18 as feature extractor
class ResNetModel(nn.Module):
    def __init__(self):
        super(ResNetModel, self).__init__()
        self.resnet = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
        self.resnet.fc = nn.Linear(self.resnet.fc.in_features, 1)
    
    def forward(self, x):
        batch_size, num_frames, C, H, W = x.size()
        x = x.view(-1, C, H, W)  # Merge batch and frames
        x = self.resnet(x)
        x = x.view(batch_size, num_frames)
        x = x.mean(dim=1).unsqueeze(1)
        return x

# Define transformations with augmentation
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load dataset
real_dir = r'C:\Users\Harshita\Desktop\deepfake\FF++\real'
fake_dir = r'C:\Users\Harshita\Desktop\deepfake\FF++\fake'
dataset = VideoDataset(real_dir, fake_dir, transform=transform)
train_dataset, val_dataset = train_test_split(dataset, test_size=0.2, random_state=42, stratify=dataset.labels)

# Create dataloaders
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)

# Initialize model, loss function, optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ResNetModel().to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.5)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device).float().unsqueeze(1)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    scheduler.step()
    print(f"Epoch {epoch+1}, Loss: {running_loss / len(train_loader)}")

    # Validation
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device).float().unsqueeze(1)
            outputs = model(inputs)
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f"Validation Accuracy: {100 * correct / total:.2f}%")

# Save model
torch.save(model.state_dict(), 'fake_video_detection_model.pth')


Epoch 1, Loss: 0.5949419286440719
Validation Accuracy: 73.86%
Epoch 2, Loss: 0.4153774505989118
Validation Accuracy: 68.18%
Epoch 3, Loss: 0.3506501540541649
Validation Accuracy: 75.00%
Epoch 4, Loss: 0.2751523981040174
Validation Accuracy: 67.05%
Epoch 5, Loss: 0.30833838372067973
Validation Accuracy: 76.14%
Epoch 6, Loss: 0.23656564934009855
Validation Accuracy: 75.00%
Epoch 7, Loss: 0.21047663815658202
Validation Accuracy: 77.27%
Epoch 8, Loss: 0.182336325884204
Validation Accuracy: 77.27%
Epoch 9, Loss: 0.17252596611665053
Validation Accuracy: 75.00%
Epoch 10, Loss: 0.17277906284752217
Validation Accuracy: 76.14%


In [4]:
import gradio as gr


c:\Users\Harshita\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
# Gradio Interface
# Ensure this function exists BEFORE predict_video()
def extract_frames(video_path, num_frames=16):
    cap = cv2.VideoCapture(video_path)
    frames = []
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_indices = np.linspace(0, max(1, total_frames) - 1, num_frames, dtype=int)

    for i in frame_indices:
        cap.set(cv2.CAP_PROP_POS_FRAMES, i)
        ret, frame = cap.read()
        if ret:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        else:
            frame = np.zeros((224, 224, 3), dtype=np.uint8)  # Black frame fallback
        frames.append(frame)
    cap.release()
    return frames

def predict_video(video):
    video_path = video  # Ensure path handling
    frames = extract_frames(video_path)  
    frames = [transform(frame) for frame in frames]
    frames = torch.stack(frames).unsqueeze(0).to(device)  # Ensure correct shape
    
    model.eval()
    with torch.no_grad():
        output = model(frames)
        probability = torch.sigmoid(output).item()
    
    confidence = round(probability * 100, 2)
    if probability > 0.4:
        return f"Fake Video ({confidence}% confidence)"
    else:
        return f"Real Video ({100 - confidence}% confidence)"

def predict_video(video):
    video_path = video if isinstance(video, str) else video.name  # Ensure path handling
    frames = extract_frames(video_path)
    frames = [transform(frame) for frame in frames]
    frames = torch.stack(frames).unsqueeze(0).to(device)  # Ensure correct shape
    model.eval()
    with torch.no_grad():
        output = model(frames)
        probability = torch.sigmoid(output).item()
    confidence = round(probability * 100, 2)
    if probability > 0.5:
        return f"Fake Video ({confidence}% confidence)"
    else:
        return f"Real Video ({100 - confidence}% confidence)"

demo = gr.Interface(
    fn=predict_video,
    inputs=gr.Video(label="Upload Video"),
    outputs="text",
    title="Deepfake Video Detector",
    description="Upload a video to check if it's real or fake."
)

demo.launch()

* Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.


Using existing dataset file at: .gradio\flagged\dataset2.csv


Traceback (most recent call last):
  File "c:\Users\Harshita\AppData\Local\Programs\Python\Python311\Lib\site-packages\gradio\queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Harshita\AppData\Local\Programs\Python\Python311\Lib\site-packages\gradio\route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Harshita\AppData\Local\Programs\Python\Python311\Lib\site-packages\gradio\blocks.py", line 2088, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Harshita\AppData\Local\Programs\Python\Python311\Lib\site-packages\gradio\blocks.py", line 1635, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Harshita\